# Subcellular location classification model via XGBoost

## Import package

In [1]:
from Bio import SeqIO
from Bio.SeqUtils import ProtParam

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import xgboost as xgb
from sklearn.metrics import log_loss
from sklearn.cross_validation import StratifiedKFold
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

/Users/lingpae/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/lingpae/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## Loading, Preprocessing and Feature engineering

In [ ]:
#Read data
def read_fasta(filenames):
    target = filenames.split('.')[0]
    seq_id = []
    seq = []
    len_seq = []
    mole_weight = []
    aromatic = []
    inst_idx = []
    hydro = []
    iso_elec = []
    sec_str_frac_1 = []
    sec_str_frac_2 = []
    sec_str_frac_3 = []
    nls_f = []
    
    f_seq = []
    f_mole_weight = []
    f_aromatic = []
    f_inst_idx = []
    f_hydro = []
    f_iso_elec = []
    f_sec_str_frac_1 = []
    f_sec_str_frac_2 = []
    f_sec_str_frac_3 = []
    
    l_seq = []
    l_mole_weight = []
    l_aromatic = []
    l_inst_idx = []
    l_hydro = []
    l_iso_elec = []
    l_sec_str_frac_1 = []
    l_sec_str_frac_2 = []
    l_sec_str_frac_3 = []
    
    for seq_record in SeqIO.parse(filenames, "fasta"):
        seq_id.append(seq_record.id)
        
        prep_seq = str(seq_record.seq)
        prep_seq = prep_seq.replace("X", "")
        prep_seq = prep_seq.replace("U", "C")
        prep_seq = prep_seq.replace("B", "N")
        prep_seq = prep_seq.replace('Z', 'Q')
        
        seq.append(prep_seq)
        len_seq.append(len(seq_record))
        temp = ProtParam.ProteinAnalysis(prep_seq) 
        
        mole_weight.append(temp.molecular_weight())
        aromatic.append(temp.aromaticity())
        inst_idx.append(temp.instability_index())
        hydro.append(temp.gravy())
        iso_elec.append(temp.isoelectric_point())
        sec_str_frac_1.append(temp.secondary_structure_fraction()[0])
        sec_str_frac_2.append(temp.secondary_structure_fraction()[1])
        sec_str_frac_3.append(temp.secondary_structure_fraction()[2])
        nls_f.append(nls_flag(prep_seq))
        
        temp_f_seq = prep_seq[0:50]
        f_seq.append(temp_f_seq)
        temp = ProtParam.ProteinAnalysis(temp_f_seq)

        f_mole_weight.append(temp.molecular_weight())
        f_aromatic.append(temp.aromaticity())
        f_inst_idx.append(temp.instability_index())
        f_hydro.append(temp.gravy())
        f_iso_elec.append(temp.isoelectric_point())
        f_sec_str_frac_1.append(temp.secondary_structure_fraction()[0])
        f_sec_str_frac_2.append(temp.secondary_structure_fraction()[1])
        f_sec_str_frac_3.append(temp.secondary_structure_fraction()[2])
        
        temp_l_seq = prep_seq[-50:]
        l_seq.append(temp_l_seq)
        temp = ProtParam.ProteinAnalysis(temp_l_seq)
 
        l_mole_weight.append(temp.molecular_weight())
        l_aromatic.append(temp.aromaticity())
        l_inst_idx.append(temp.instability_index())
        l_hydro.append(temp.gravy())
        l_iso_elec.append(temp.isoelectric_point())
        l_sec_str_frac_1.append(temp.secondary_structure_fraction()[0])
        l_sec_str_frac_2.append(temp.secondary_structure_fraction()[1])
        l_sec_str_frac_3.append(temp.secondary_structure_fraction()[2])

    df = pd.DataFrame({ 'seq' : seq, 'len_seq' : len_seq, 
                       'mole_weight' : mole_weight,
                       'aromatic' : aromatic, 'inst_idx' : inst_idx, 'hydro' : hydro,
                       'iso_elec' : iso_elec,
                       'sec_str_frac_1' : sec_str_frac_1, 'sec_str_frac_2' : sec_str_frac_2,
                       'sec_str_frac_3' : sec_str_frac_3, 'nls_f' : nls_f,
                       
                       'f_seq' : f_seq, 'f_mole_weight' : f_mole_weight,
                       'f_aromatic' : f_aromatic, 'f_inst_idx' : f_inst_idx, 'f_hydro' : f_hydro,
                       'f_iso_elec' : f_iso_elec,
                       'f_sec_str_frac_1' : f_sec_str_frac_1, 'f_sec_str_frac_2' : f_sec_str_frac_2,
                       'f_sec_str_frac_3' : f_sec_str_frac_3,
                       
                       'l_seq' : l_seq, 'l_mole_weight' : l_mole_weight,
                       'l_aromatic' : l_aromatic, 'l_inst_idx' : l_inst_idx, 'l_hydro' : l_hydro,
                       'l_iso_elec' : l_iso_elec,
                       'l_sec_str_frac_1' : l_sec_str_frac_1, 'l_sec_str_frac_2' : l_sec_str_frac_2,
                       'l_sec_str_frac_3' : l_sec_str_frac_3
                      },index = seq_id)
    df['y'] = target
    return df

def data_pipeline(dat):
    for a in amino_dic:
        dat['num_' + a] = dat['seq'].str.count(a)
        dat['p_' + a] = dat['seq'].str.count(a)/dat['len_seq']
        dat['f_num_' + a] = dat['f_seq'].str.count(a)
        dat['f_p_' + a] = dat['f_seq'].str.count(a)/50
        dat['l_num_' + a] = dat['l_seq'].str.count(a)
        dat['l_p_' + a] = dat['l_seq'].str.count(a)/50
    for di in di_dic:
        dat['num_' + di] = dat['seq'].str.count(di)
        dat['f_num_' + di] = dat['f_seq'].str.count(di)
        dat['l_num_' + di] = dat['f_seq'].str.count(di)
    X = dat
    X = X.drop(['y','seq', 'f_seq', 'l_seq'], axis = 1)
    Y = dat['y']
    return X, Y

In [ ]:
nls_db = pd.read_csv('nuclear.csv')
nls_sig_list = nls_db['Sequence']

cyto = read_fasta('cyto.fasta')
mito = read_fasta('mito.fasta')
nucleus = read_fasta('nucleus.fasta')
secreted = read_fasta('secreted.fasta')

df = pd.concat([cyto, mito, nucleus, secreted])

amino_dic = ['I','L','V','F','M','C','A','G','P','T','S','Y','W','Q','N','H','E','D','K','R']
di_dic = ['II','IL','IV','IF','IM','IC','IA','IG','IP','IT','IS','IY','IW','IQ','IN','IH','IE','ID','IK','IR',
          'LI','LL','LV','LF','LM','LC','LA','LG','LP','LT','LS','LY','LW','LQ','LN','LH','LE','LD','LK','LR',
          'VI','VL','VV','VF','VM','VC','VA','VG','VP','VT','VS','VY','VW','VQ','VN','VH','VE','VD','VK','VR',
          'FI','FL','FV','FF','FM','FC','FA','FG','FP','FT','FS','FY','FW','FQ','FN','FH','FE','FD','FK','FR',
          'MI','ML','MV','MF','MM','MC','MA','MG','MP','MT','MS','MY','MW','MQ','MN','MH','ME','MD','MK','MR',
          'CI','CL','CV','CF','CM','CC','CA','CG','CP','CT','CS','CY','CW','CQ','CN','CH','CE','CD','CK','CR',
          'AI','AL','AV','AF','AM','AC','AA','AG','AP','AT','AS','AY','AW','AQ','AN','AH','AE','AD','AK','AR',
          'GI','GL','GV','GF','GM','GC','GA','GG','GP','GT','GS','GY','GW','GQ','GN','GH','GE','GD','GK','GR',
          'PI','PL','PV','PF','PM','PC','PA','PG','PP','PT','PS','PY','PW','PQ','PN','PH','PE','PD','PK','PR',
          'TI','TL','TV','TF','TM','TC','TA','TG','TP','TT','TS','TY','TW','TQ','TN','TH','TE','TD','TK','TR',
          'SI','SL','SV','SF','SM','SC','SA','SG','SP','ST','SS','SY','SW','SQ','SN','SH','SE','SD','SK','SR',
          'YI','YL','YV','YF','YM','YC','YA','YG','YP','YT','YS','YY','YW','YQ','YN','YH','YE','YD','YK','YR',
          'WI','WL','WV','WF','WM','WC','WA','WG','WP','WT','WS','WY','WW','WQ','WN','WH','WE','WD','WK','WR',
          'QI','QL','QV','QF','QM','QC','QA','QG','QP','QT','QS','QY','QW','QQ','QN','QH','QE','QD','QK','QR',
          'NI','NL','NV','NF','NM','NC','NA','NG','NP','NT','NS','NY','NW','NQ','NN','NH','NE','ND','NK','NR',
          'HI','HL','HV','HF','HM','HC','HA','HG','HP','HT','HS','HY','HW','HQ','HN','HH','HE','HD','HK','HR',
          'EI','EL','EV','EF','EM','EC','EA','EG','EP','ET','ES','EY','EW','EQ','EN','EH','EE','ED','EK','ER',
          'DI','DL','DV','DF','DM','DC','DA','DG','DP','DT','DS','DY','DW','DQ','DN','DH','DE','DD','DK','DR',
          'KI','KL','KV','KF','KM','KC','KA','KG','KP','KT','KS','KY','KW','KQ','KN','KH','KE','KD','KK','KR',
          'RI','RL','RV','RF','RM','RC','RA','RG','RP','RT','RS','RY','RW','RQ','RN','RH','RE','RD','RK','RR']

In [ ]:
X, Y = data_pipeline(df)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

## Training and Tuning Model

In [ ]:
#Random Search Cross validation
clf = xgb.XGBClassifier()

param_grid = {
        'silent': [False],
        'max_depth': [5, 7, 10, 11, 12, 13, 14, 15],
        'learning_rate': [0.001,0.01, 0.05, 0.1],
        'subsample': [0.7, 0.8, 0.9],
        'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0, 13.0],
        'gamma': [0,0.1,0.2, 0.25, 0.5],
        'n_estimators': [70, 100,400, 500, 1000],
        'objective' : 'multi:softmax'}

In [ ]:
fit_params_A = {'eval_metric': 'mlogloss',
              'early_stopping_rounds': 10,
              'eval_set': [(X_val, y_val)]}
clf = RandomizedSearchCV(clf, param_grid, n_iter=20,
                            n_jobs=5, verbose=2, cv=5,
                            fit_params=fit_params_A,
                            random_state=42)
#                             scoring='roc_auc')


clf.fit(X_train, y_train)

In [ ]:
best_score = model_A.best_score_
best_params = model_A.best_params_
print("Best score: {}".format(best_score))
print("Best params: ")
for param_name in sorted(best_params.keys()):
    print('%s: %r' % (param_name, best_params[param_name]))

In [ ]:
#Final Model
clf = xgb.XGBClassifier(colsample_bylevel =  0.9,
                        colsample_bytree= 0.6,
                        gamma= 0.5,
                        learning_rate= 0.1,
                        max_depth= 13,
                        min_child_weight= 10.0,
                        n_estimators= 500,
                        objective = 'multi:softmax',
                        silent= False,
                        subsample= 0.7,
                        random_state=42)
clf.fit(X_train, y_train,eval_metric = 'mlogloss',
              early_stopping_rounds = 10,
              eval_set = [(X_val, y_val)])

In [ ]:
print('Accuracy of XGBoost classifier on test set: {:.2f}'.format(clf.score(X_test, y_test)))

## Model Evaluation

In [ ]:
y_pred = clf.predict(X_test)

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)

np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
class_names = ['cyto', 'mito', 'nucleus', 'secreted']
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

plt.show()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred,  
             target_names= ['cyto','mito','nucleus','secreted']))

In [ ]:
#Feature importance
trail = pd.DataFrame(clf.feature_importances_,X_train.columns.values)
trail.sort_values(by=0, ascending=False)[0:50]

# Prediction Section

In [ ]:
blind = read_fasta('blind.fasta')
blind_X, _ = data_pipeline(blind)

In [ ]:
pd.DataFrame({'predict' : clf.predict(blind_X),
              'Confident' : np.max(clf.predict_proba(blind_X), axis = 1)
             },index=blind_X.index)